In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import glob
import tqdm as tqdm
import random
import tensorflow as tf
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
column_names=["file_path","label"]
train=pd.DataFrame(columns=column_names)
test=pd.DataFrame(columns=column_names)

In [4]:
for df,folder in zip([train,test],['train','test']):
    file_paths=glob.glob("/kaggle/input/cifar10/cifar10/"+folder+"/*/*")
    df['file_path']=file_paths
    labels=[]
    for file_path in tqdm.tqdm(file_paths):
        label=file_path.split('_')[-1].split('.')[0]
        labels.append(label)
    df['label']=labels

100%|██████████| 10000/10000 [00:00<00:00, 1217398.78it/s]


In [5]:
train

,file_path,label
0,/kaggle/input/cifar10/cifar10/train/airplane/2...,airplane
1,/kaggle/input/cifar10/cifar10/train/airplane/2...,airplane
2,/kaggle/input/cifar10/cifar10/train/airplane/4...,airplane
3,/kaggle/input/cifar10/cifar10/train/airplane/1...,airplane
4,/kaggle/input/cifar10/cifar10/train/airplane/4...,airplane
...,...,...
49995,/kaggle/input/cifar10/cifar10/train/deer/41608...,deer
49996,/kaggle/input/cifar10/cifar10/train/deer/6785_...,deer
49997,/kaggle/input/cifar10/cifar10/train/deer/17777...,deer
49998,/kaggle/input/cifar10/cifar10/train/deer/9202_...,deer


In [6]:
def final_data(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[90,120])
    image = image/255
    return image,label

In [7]:
def transform(df,training=False):
    labels=pd.get_dummies(df.label)
    preliminary_data=list(zip(df['file_path'],labels.values.tolist()))
    if(training==True):
        random.shuffle(preliminary_data)
    paths,labels=zip(*preliminary_data)
    data=tf.data.Dataset.from_tensor_slices((list(paths),list(labels)))
    data=data.map(final_data)
    return data

In [8]:
train_data=transform(train,training=True).prefetch(tf.data.AUTOTUNE)

In [9]:
test_data=transform(test)

In [10]:
test_data=test_data.take(10000)

In [11]:
val_size=round(0.1*50000)
train_size=round(0.9*50000)
train_data=train_data.take(train_size)
val_data=train_data.skip(train_size)
val_data=train_data.take(val_size)

In [12]:
train_data

<_TakeDataset element_spec=(TensorSpec(shape=(90, 120, 3), dtype=tf.float32, name=None), TensorSpec(shape=(10,), dtype=tf.int32, name=None))>

In [13]:
mirrored_strategy=tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1"])

In [14]:
with mirrored_strategy.scope():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(30,(5,5),strides=(1,1),padding='valid',
                                     activation='relu',input_shape=(90,120,3)))
    #1
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
    #2
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(20,(3,3),strides=(1,1),padding='valid',activation='relu'))
    #3
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
    #4
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.GroupNormalization(groups=3))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
    #5
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(10,(3,3),strides=(1,1),padding='valid',activation='relu'))
    #6
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Normalization())
    model.add(tf.keras.layers.Dense(256,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(128,activation='relu'))
    model.add(tf.keras.layers.Dense(10,activation='softmax'))
    model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 86, 116, 30)       2280      
                                                                 
 batch_normalization (BatchN  (None, 86, 116, 30)      120       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 84, 114, 30)       8130      
                                                                 
 batch_normalization_1 (Batc  (None, 84, 114, 30)      120       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 42, 57, 30)       0         
 )                                                               
                                                        

In [16]:
with mirrored_strategy.scope():
    checkpoint= tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/cifar_10_classifier_4.h5',
                                                   save_weights_only=False,monitor='val_accuracy',
                                                   save_best_only=True,save_freq="epoch",)
    early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,
                                                     restore_best_weights=True)
    def lr_scheduler(epoch,lr,epochs=50):
        initial=1e-3
        if epoch<epochs*0.1:
            return initial
        elif epoch>epochs*0.1 and epoch<epochs*0.25:
            lr*=tf.math.exp(-0.1)
            return lr
        else:
            lr*=tf.math.exp(-0.008)
            return lr
    lr_scheduling=tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [17]:
with mirrored_strategy.scope():
    history = model.fit(train_data.batch(150),epochs=50,validation_data=val_data.batch(150),
                        callbacks=[checkpoint,early_stopping,lr_scheduling],shuffle=True)

Epoch 1/50
300/300 [==============================] - 843s 3s/step - loss: 1.4675 - accuracy: 0.4756 - val_loss: 3.4803 - val_accuracy: 0.2408 - lr: 0.0010
Epoch 2/50
300/300 [==============================] - 829s 3s/step - loss: 1.0418 - accuracy: 0.6305 - val_loss: 0.9561 - val_accuracy: 0.6610 - lr: 0.0010
Epoch 3/50
300/300 [==============================] - 865s 3s/step - loss: 0.8244 - accuracy: 0.7116 - val_loss: 0.8193 - val_accuracy: 0.7130 - lr: 0.0010
Epoch 4/50
300/300 [==============================] - 825s 3s/step - loss: 0.6427 - accuracy: 0.7760 - val_loss: 0.8241 - val_accuracy: 0.7320 - lr: 0.0010
Epoch 5/50
300/300 [==============================] - 826s 3s/step - loss: 0.4807 - accuracy: 0.8339 - val_loss: 0.5328 - val_accuracy: 0.8120 - lr: 0.0010
Epoch 6/50
300/300 [==============================] - 823s 3s/step - loss: 0.3661 - accuracy: 0.8753 - val_loss: 0.5014 - val_accuracy: 0.8240 - lr: 9.9203e-04
Epoch 7/50
300/300 [==============================] - 826s 3

In [18]:
model=tf.keras.models.load_model("/kaggle/working/cifar_10_classifier_4.h5")

In [19]:
with mirrored_strategy.scope():
    predictions=model.predict(test_data.batch(10000))

1/1 [==============================] - 162s 162s/step


In [20]:
def outputs(x):
    a = np.zeros(x.shape)
    a[np.where(x==np.max(x))] = 1
    return a

In [21]:
for i in range(len(predictions)):
     predictions[i]=outputs(predictions[i])

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
y_test=np.concatenate([y for x, y in test_data.batch(10000)],axis=0)

In [24]:
accuracy_score(predictions,y_test)

0.649